In [2]:
print "Importing packages"

import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn as learn
import xgboost
import pandas as pd
import numpy as np
import re


Importing packages


Third attempt at the DonorsChoose Dataset. Very basic stacking model.
Anisotropic's notebook on Stacking\ensembling was really useful for creating this.

In [3]:
#reading data

train= pd.read_csv("/home/souravg/train.csv")

test=pd.read_csv("/home/souravg/test.csv") 

resource= pd.read_csv("/home/souravg/resources.csv")

print train.head(7)

print resource.head(7)


joined= pd.merge(train,resource,how='inner',on='id',sort=False,)

joined_test=pd.merge(test,resource,how='inner',on='id',sort=False,)


print joined.columns

print joined.head(5)

#train.school_state.astype("category")

#train.project_grade_category.astype("category")

#train.project_subject_categories.astype("category")

#train.project_subject_subcategories.astype("category")




/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


        id                        teacher_id teacher_prefix school_state  \
0  p036502  484aaf11257089a66cfedc9461c6bd0a            Ms.           NV   
1  p039565  df72a3ba8089423fa8a94be88060f6ed           Mrs.           GA   
2  p233823  a9b876a9252e08a55e3d894150f75ba3            Ms.           UT   
3  p185307  525fdbb6ec7f538a48beebaa0a51b24f            Mr.           NC   
4  p013780  a63b5547a7239eae4c1872670848e61a            Mr.           CA   
5  p063374  403c6783e9286e51ab318fba40f8d729           Mrs.           DE   
6  p103285  4e156c5fb3eea2531601c8736f3751a7           Mrs.           MO   

  project_submitted_datetime project_grade_category  \
0        2016-11-18 14:45:59          Grades PreK-2   
1        2017-04-26 15:57:28             Grades 3-5   
2        2017-01-01 22:57:44             Grades 3-5   
3        2016-08-12 15:42:11             Grades 3-5   
4        2016-08-06 09:09:11             Grades 6-8   
5        2016-11-05 10:01:51          Grades PreK-2   
6     

In [4]:
joined['Total_sum']= joined.quantity*joined.price
joined_test['Total_sum']= joined_test.quantity*joined_test.price
print joined.head(3)

print '-------------------------------'

print joined_test.head(3)


        id                        teacher_id teacher_prefix school_state  \
0  p036502  484aaf11257089a66cfedc9461c6bd0a            Ms.           NV   
1  p036502  484aaf11257089a66cfedc9461c6bd0a            Ms.           NV   
2  p039565  df72a3ba8089423fa8a94be88060f6ed           Mrs.           GA   

  project_submitted_datetime project_grade_category  \
0        2016-11-18 14:45:59          Grades PreK-2   
1        2016-11-18 14:45:59          Grades PreK-2   
2        2017-04-26 15:57:28             Grades 3-5   

          project_subject_categories project_subject_subcategories  \
0                Literacy & Language                      Literacy   
1                Literacy & Language                      Literacy   
2  Music & The Arts, Health & Sports  Performing Arts, Team Sports   

              project_title  \
0  Super Sight Word Centers   
1  Super Sight Word Centers   
2    Keep Calm and Dance On   

                                     project_essay_1  \
0  Most of m

In [5]:
sumdf= joined.groupby('id',as_index=False).agg({'Total_sum':'sum'})

sumdf_test=joined_test.groupby('id',as_index=False).agg({'Total_sum':'sum'})

print sumdf.head(3)
print '-----------------------'
print sumdf_test.head(3)

#print joined.Total_sum.head(3)

        id  Total_sum
0  p000002     630.28
1  p000003     298.97
2  p000004    1126.22
-----------------------
        id  Total_sum
0  p000001     833.63
1  p000005     702.31
2  p000011     251.80


In [6]:
final_train= pd.merge(train,sumdf,on='id',how='inner',sort=False,)
final_test=pd.merge(test,sumdf_test,on='id',how='inner',sort=False,)

print final_train.head(3)
print '-------------------'
print final_test.head(3)

        id                        teacher_id teacher_prefix school_state  \
0  p036502  484aaf11257089a66cfedc9461c6bd0a            Ms.           NV   
1  p039565  df72a3ba8089423fa8a94be88060f6ed           Mrs.           GA   
2  p233823  a9b876a9252e08a55e3d894150f75ba3            Ms.           UT   

  project_submitted_datetime project_grade_category  \
0        2016-11-18 14:45:59          Grades PreK-2   
1        2017-04-26 15:57:28             Grades 3-5   
2        2017-01-01 22:57:44             Grades 3-5   

            project_subject_categories  \
0                  Literacy & Language   
1    Music & The Arts, Health & Sports   
2  Math & Science, Literacy & Language   

            project_subject_subcategories             project_title  \
0                                Literacy  Super Sight Word Centers   
1            Performing Arts, Team Sports    Keep Calm and Dance On   
2  Applied Sciences, Literature & Writing     Lets 3Doodle to Learn   

                    

The cell above read the train data, converted appropriate parameters to categorical type & did some group by counts. From the results it seems like California, New York, Texas & Florida have the largest number of projects.
Grades PreK-2 had the most number of projects, & the number of projects decrease monotonically with increasing age of students. Grades 9-12 have the fewest number of projects. I must say that this came as a surprise, I would hae thought seniors would need more funding.
There seems to be the following project categories:-

Applied Learning
Health & Sports
History & Civics
Math & Science
Music & The Arts
Special Needs
Warmth, Care & Hunger

But the majority of projects are tagged with multiple categories, hence there are lot more different project categories in the data (combinations of the base categories).

Math & Science, Literacy & Language & Health & Sport seem to be leaders in terms of project count.

In [18]:
train=final_train
test=final_test

copy=pd.DataFrame(pd.concat([train,test]))
print copy.head(3)




copy["school_state"]=copy["school_state"].astype('category')
copy["project_grade_category"]=copy["project_grade_category"].astype('category')
copy["project_subject_categories"]=copy["project_subject_categories"].astype('category')
copy["project_subject_subcategories"]=copy["project_subject_subcategories"].astype('category')

ftest=test.drop([u'id', u'teacher_id', u'teacher_prefix',u'project_title', u'project_essay_1', u'project_essay_2',
       u'project_essay_3', u'project_essay_4', u'project_resource_summary','project_submitted_datetime'],1)

print ftest.head(4)

#print target.head(5)

ftest["school_state"]=ftest["school_state"].astype('category')
ftest["project_grade_category"]=ftest["project_grade_category"].astype('category')
ftest["project_subject_categories"]=ftest["project_subject_categories"].astype('category')
ftest["project_subject_subcategories"]=ftest["project_subject_subcategories"].astype('category')

print ftest["school_state"].describe()
print ftest["project_grade_category"].describe()
print ftest["project_subject_categories"].describe()
print ftest["project_subject_subcategories"].describe()
print '*************************************printing category mapping'
l=list(zip(ftest.school_state.cat.codes, ftest.school_state.cat.categories))
for i in l:
    print i

print '*****************Converting categoricals to dummies*****************'

ftest=pd.get_dummies(ftest,prefix=['state','grade','categories','subcategories'])

print '***************************************'



   Total_sum       id                                    project_essay_1  \
0     899.94  p036502  Most of my kindergarten students come from low...   
1     400.00  p039565  Our elementary school is a culturally rich sch...   
2     469.99  p233823  Hello;\r\nMy name is Mrs. Brotherton. I teach ...   

                                     project_essay_2 project_essay_3  \
0  I currently have a differentiated sight word c...             NaN   
1  We strive to provide our diverse population of...             NaN   
2  We are looking to add some 3Doodler to our cla...             NaN   

  project_essay_4 project_grade_category  project_is_approved  \
0             NaN          Grades PreK-2                  1.0   
1             NaN             Grades 3-5                  0.0   
2             NaN             Grades 3-5                  1.0   

                            project_resource_summary  \
0  My students need 6 Ipod Nano's to create and d...   
1  My students need matching shir

In [16]:
state_sum=train.groupby("school_state").size()

#print state_sum

ftrain=train.drop([u'id', u'teacher_id', u'teacher_prefix',u'project_title', u'project_essay_1', u'project_essay_2',
       u'project_essay_3', u'project_essay_4', u'project_resource_summary','project_submitted_datetime'],1)

print ftrain.head(4)

target=ftrain.project_is_approved

ftrain=ftrain.drop(['project_is_approved'],1)

ftrain["school_state"]=ftrain["school_state"].astype('category')
ftrain["project_grade_category"]=ftrain["project_grade_category"].astype('category')
ftrain["project_subject_categories"]=ftrain["project_subject_categories"].astype('category')
ftrain["project_subject_subcategories"]=ftrain["project_subject_subcategories"].astype('category')

print '**************************printing school categories************'
l=list(zip(ftrain.school_state.cat.codes, ftrain.school_state.cat.categories))
for i in l:
    print i
print '*****************************************'

copy=pd.DataFrame(pd.concat([ftrain,ftest]))
print copy.head(3)
l=list(zip(copy.school_state.cat.codes, copy.school_state.cat.categories))
for i in l:
    print i
print ftrain["school_state"].describe()
print ftrain["project_grade_category"].describe()
print ftrain["project_subject_categories"].describe()
print ftrain["project_subject_subcategories"].describe()

print ftrain["project_subject_subcategories"].head(10)

print '**************** Converting categoricals to dummies************'

ftrain=pd.get_dummies(ftrain,prefix=['state','grade','categories','subcategories'])

print '***************************************************'

print ftrain.head(2)

print '========================================== Normalization of project count & total sum'

projectcount_mean=ftrain['teacher_number_of_previously_posted_projects'].mean()
projectcount_sd=ftrain['teacher_number_of_previously_posted_projects'].std()

totalsum_mean=ftrain['Total_sum'].mean()
totalsum_sd=ftrain['Total_sum'].std()

print projectcount_mean,' ',projectcount_sd

ftrain['Total_sum']=(ftrain['Total_sum']-totalsum_mean)/totalsum_sd
ftrain['teacher_number_of_previously_posted_projects']= (ftrain['teacher_number_of_previously_posted_projects']-projectcount_mean)/projectcount_sd



ftest['Total_sum']=(ftest['Total_sum']-totalsum_mean)/totalsum_sd
ftest['teacher_number_of_previously_posted_projects']= (ftest['teacher_number_of_previously_posted_projects']-projectcount_mean)/projectcount_sd


print '-------------------------------- Normalization done'

print ftrain.head(3)

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn import ensemble
from sklearn.neural_network import MLPClassifier

abclf= ensemble.AdaBoostClassifier(n_estimators=1000,)
rfclf= ensemble.RandomForestClassifier(n_estimators=1000,criterion='entropy',max_depth=5,warm_start=True,)
gbclf= ensemble.GradientBoostingClassifier(n_estimators=1000,max_depth=4,min_samples_split=4,subsample=.3)

clf= ensemble.RandomForestClassifier(n_estimators=500,criterion='entropy',max_depth=3,warm_start=True,)
#clf = AdaBoostClassifier(n_estimators=500, learning_rate= 0.4)
#clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(200, 100), random_state=1,max_iter=1000)
print "Hola"



clf.fit(ftrain,target)


#abclf.fit(ftrain, target)
#rfclf.fit(ftrain, target)
#gbclf.fit(ftrain, target)

#print 'Feature importance from adaboost='
#print abclf.feature_importances_

#print 'Feature importance from random forest='
#print rfclf.feature_importances_

#print 'Feature importance from gradient boosting='
#print gbclf.feature_importances_

#tabresult=abclf.predict_proba(ftrain)
#trfresult=rfclf.predict_proba(ftrain)
#tgbresult=gbclf.predict_proba(ftrain)


#abresult=abclf.predict_proba(ftest)
#rfresult=rfclf.predict_proba(ftest)
#gbresult=gbclf.predict_proba(ftest)

print "finito"

  school_state project_grade_category           project_subject_categories  \
0           NV          Grades PreK-2                  Literacy & Language   
1           GA             Grades 3-5    Music & The Arts, Health & Sports   
2           UT             Grades 3-5  Math & Science, Literacy & Language   
3           NC             Grades 3-5                      Health & Sports   

            project_subject_subcategories  \
0                                Literacy   
1            Performing Arts, Team Sports   
2  Applied Sciences, Literature & Writing   
3                       Health & Wellness   

   teacher_number_of_previously_posted_projects  project_is_approved  \
0                                            26                    1   
1                                             1                    0   
2                                             5                    1   
3                                            16                    0   

   Total_sum  
0     8

AttributeError: Can only use .cat accessor with a 'category' dtype

In [42]:

result=clf.predict_proba(ftest)

r_df=pd.DataFrame(dict(id=test.id,project_is_approved=result[:,1]))

print r_df.shape
print r_df.head(30)
#print rfclf.feature_importances_


r_df.to_csv('/home/souravg/output5.csv',index=False)


ValueError: shapes (78035,498) and (515,200) not aligned: 498 (dim 1) != 515 (dim 0)

[[ 0.16478707  0.83521293]
 [ 0.13464939  0.86535061]
 [ 0.18761641  0.81238359]
 [ 0.14355027  0.85644973]
 [ 0.17657868  0.82342132]
 [ 0.15248022  0.84751978]
 [ 0.13040103  0.86959897]
 [ 0.14690793  0.85309207]
 [ 0.10957476  0.89042524]
 [ 0.14672544  0.85327456]
 [ 0.13865819  0.86134181]
 [ 0.15728328  0.84271672]
 [ 0.16674882  0.83325118]
 [ 0.13409432  0.86590568]
 [ 0.12280192  0.87719808]
 [ 0.11945113  0.88054887]
 [ 0.14228784  0.85771216]
 [ 0.19036895  0.80963105]
 [ 0.14258076  0.85741924]
 [ 0.13383068  0.86616932]
 [ 0.17531093  0.82468907]
 [ 0.16300933  0.83699067]
 [ 0.17815078  0.82184922]
 [ 0.17188554  0.82811446]
 [ 0.1812131   0.8187869 ]
 [ 0.17091193  0.82908807]
 [ 0.16154056  0.83845944]
 [ 0.08719656  0.91280344]
 [ 0.14170426  0.85829574]
 [ 0.09523673  0.90476327]]
0     p233245
1     p096795
2     p236235
3     p233680
4     p171879
5     p016071
6     p099906
7     p200236
8     p129452
9     p186652
10    p257992
11    p055177
12    p069502
13    p